# The Beermile

## Retrieve Data

In [1]:
import sys
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [2]:
url_base = 'https://www.beermile.com/display/'
max_event_id = 151381

In [ ]:
%%time

# Find which events are actually races
is_event = 0
not_event = 0


# Look for all events by numerical ID
for i in range(1+410, max_event_id):
    # Website uses sequential 
    event = f'event_{i}'
    url = f'{url_base}{event}'
    result = requests.get(url)
    page = urlopen(result.url).read()
    a = bs(page, 'html.parser')
    
    eventType = False
    if (a.title.string == 'Race Not Available'):
        not_event += 1
        with open('not_events.txt', 'a') as g:
            g.write(f'{i}\n')
    else:
        is_event += 1
        eventType = True
        with open('events.txt', 'a') as f:
            f.write(f'{i}\n')
            
    print(f'\r #{i:06d} Events: {is_event:<6} - Not Events: {not_event:<6}', end='') 
    sys.stdout.flush()


 #022809 Events: 2375   - Not Events: 20024 

## Clean Data Into DataFrame

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
def dfFromTable(table_soup):
    is_header = True
    pd_cols = []
    df = None
    
    rows = table_soup.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        # TODO: Determine how to "clean" string
        cols = [ele.text.strip() for ele in cols]
        row = [ele for ele in cols if ele]
        # First row will be the names for the columns
        if (is_header):
            is_header = False
            # Add in place columns
            pd_cols = ['Place'] + row
            df = pd.DataFrame(columns=pd_cols)
        else:
            # Associate columns with values before insertion
            small_dict = {}
            for i in range(len(row)):
                small_dict[pd_cols[i]] = row[i]
            df = df.append(small_dict, ignore_index=True)
        
    return df